In [1]:
#!pip install ortools

import numpy as np
import datetime
import matplotlib.pyplot as plt

1. locate_call에서 np형태의 distance matrix 불러오기

In [5]:
time_matrix = np.load('../locate_call/강남역_경로매트릭스_24(분).npy')
d = time_matrix.tolist()

In [3]:
import pickle

with open("../locate_call/정류소명_리스트.pkl","rb") as f:
    name_lst = pickle.load(f)

In [19]:
"""Simple Pickup Delivery Problem (PDP) + Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = d
    data['demands'] = [0,2] + [1 for i in range(22)] 
    data['vehicle_capacities'] = [6,6,6,6]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_time = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n\n'.format(vehicle_id)
        route_time = 0
        route_time_lst = [0]
        route_load = 0
        i=0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time_lst.append(routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            
            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
            i+=1
        plan_output += '서비스 종료\n\n'
        plan_output += 'Duration of the route: {}분\n'.format((datetime.timedelta(seconds = route_time-route_time_lst[-1])))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += route_time
        total_load += route_load
    print('\n')
    print('Total Duration of all routes: {}m'.format(datetime.timedelta(seconds =total_time)))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # 1-1. Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 1-2.Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        30000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100) #이거 의미?

    # 2-1.Define cost of each node.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)

    # 2-2.Add Capacity constraint.
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    

    # Print solution on console.
    if solution:
      solution_dict =  print_solution(data, manager, routing, solution)
    else:
      print(solution)
  
if __name__ == '__main__':
    main()

Objective: 1087586
Route for vehicle 0:

 정류장 이름 : 강남역  (탑승객 수: 6, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : LH3.4단지  (탑승객 수: 5, 하차수: 1, 소요시간: 0:15:45) 
    ==>     정류장 이름 : 성지아트빌  (탑승객 수: 4, 하차수: 1, 소요시간: 0:33:35) 
    ==>     정류장 이름 : 도림사거리  (탑승객 수: 3, 하차수: 1, 소요시간: 0:18:11) 
    ==>     정류장 이름 : 홍대입구역사거리  (탑승객 수: 2, 하차수: 1, 소요시간: 0:20:53) 
    ==>     정류장 이름 : 이촌2동대림아파트.새남터성지  (탑승객 수: 1, 하차수: 1, 소요시간: 0:20:29) 
    ==>     정류장 이름 : 숭실대입구역  (탑승객 수: 0, 하차수: 1, 소요시간: 0:14:56) 
    ==>    서비스 종료

Duration of the route: 2:03:49분
Load of the route: 6

Route for vehicle 1:

 정류장 이름 : 강남역  (탑승객 수: 6, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 쌍문역  (탑승객 수: 5, 하차수: 1, 소요시간: 0:47:48) 
    ==>     정류장 이름 : 서울과학기술대학교정문앞  (탑승객 수: 4, 하차수: 1, 소요시간: 0:15:24) 
    ==>     정류장 이름 : 신내동성3차아파트  (탑승객 수: 3, 하차수: 1, 소요시간: 0:11:28) 
    ==>     정류장 이름 : 명일동국민은행.래미안솔베뉴  (탑승객 수: 2, 하차수: 1, 소요시간: 0:27:03) 
    ==>     정류장 이름 : 일자산입구.둔촌도서관  (탑승객 수: 1, 하차수: 1, 소요시간: 0:10:01) 
    ==>     정류장 이름 : 잠실역.롯데월드  

In [ ]:
coordinates = [
(456, 320), # location 0 - the depot
(228, 0),    # location 1
(912, 0),    # location 2
(0, 80),     # location 3
(114, 80),   # location 4
(570, 160),  # location 5
(798, 160),  # location 6
(342, 240),  # location 7
(684, 240),  # location 8
(570, 400),  # location 9
(912, 400),  # location 10
(114, 480),  # location 11
(228, 480),  # location 12
(342, 560),  # location 13
(684, 560),  # location 14
(0, 640),    # location 15
(798, 640)]  # location 16

X = np.array([x[0] for x in coordinates])
Y = np.array([x[1] for x in coordinates])

f, ax = plt.subplots(figsize = [8,6])
for i, txt in enumerate(coordinates):
    ax.text(X[i] + 15, Y[i]+ 15, f"{i}")

ax.plot(X, Y, 'ko', markersize=8)
ax.plot(X[0], Y[0], 'gX', markersize=15)
ax.set_title("Routing Problem Locations")
plt.show()